In [1]:
import os
import sys
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal as signal #for signal processing
import scipy.stats as stats

os.chdir("/Users/etmu9498/research/code/scripts-winter2023/")
import helper_fns_winter2023
sys.path.append(  "/Users/etmu9498/research/code/scripts-winter2023/cloud-top-height-stats")
import eyewall_metadata
import find_cloud_tops
import cloud_top_plotting


In [2]:
# helper fns

# get the power spectrum of red noise for this dataset
def create_normalized_redfit(data_length,Te):
    freq = np.arange(0,(data_length/2)+1,1)/float(data_length) # to Nyquist
    red_fit = (2 * Te)/(1 + ((2*np.pi*freq)**2)*(Te**2)) # After Hartmann 6.64, 6.91
    return red_fit/np.sum(red_fit)

# using the red noise power spectrum and a given confidence level, calculate the confidence limits for each data point
def create_f_bounds(alpha,dof,red_fit_n):
    f_ratio = stats.f.ppf(alpha,dof,200) # Note: 200 = large degree of freedom for red noise
    return f_ratio*red_fit_n

def eyewall_finder_helper(yearval, date, metadata):
    # check if this date exists... if not, give it some empty eyewall limits!
    # also account for fred am and pm cases!!
    if date == '0812':
        if fileval[11:13] == "H1":
            eyewall_limits = metadata[ yearval]['eyewall_limits'][ '0812am']
        elif fileval[11:13] == "H2":
            eyewall_limits = metadata[ yearval]['eyewall_limits'][ '0812pm']
    elif date in metadata[ yearval]['eyewall_limits'].keys():
        eyewall_limits = metadata[ yearval]['eyewall_limits'][ date]
    else:
        eyewall_limits = [ ()]

    return eyewall_limits

#Method #1
#Calculate the autocorrelation using numpy correlate lagN
def method1( t1_m, t2_m, N, lag, sigma):
    return np.correlate( t1_m, t2_m, mode='valid')/( N - lag)/( sigma**2)


In [3]:
# choose which data subset to look at (all, 2021, 2022, or a dictionary)
tc = 'all'

# save n* calculations here
nstar_total = 0
autocorrelation_sum = 0
passcount = 0

indict = {}
indict['2021'] = ['P3_20210929H2_processed.nc'] # ['P3_20210929H2_processed.nc']
# tc = indict

eye_limits='default'
crl_root_path = "/Users/etmu9498/research/data/crl-all-data-processed/"
alpha = 0.95 ## set statistical significance level
label = 'Cloud Heights'
# create an x axis
scaling = 260 / 1000
if scaling == .26:
    xlabel = "Distance from start (km)"
    freqlabel = 'Frequency (km^-1)'
    xlims, ylims = [.0001, .4], [0, .4] # [0, .01]
elif scaling == 2:
    xlabel = "Time Since Start (s)"
    freqlabel = 'Frequency (Hz)'
    xlims, ylims = [.0001, .2], [0, .4] # [0, .01]


# use a helper fn to get the relevant years and files
yearlist, filelist = helper_fns_winter2023.get_crl_datasets( tc=tc)

# print out the number of files to be used
filecount = 0
for yeari in range( len( filelist)):
    # count all the names in this year, and add to the count
    filecount += len( filelist[ yeari])
print("Number of data files to be used in analysis: " + str( filecount))

# load eyewall limits from helper function
metadata = eyewall_metadata.all_metadata( eye_limits=eye_limits)

# do this for all the datasets! years and filenames
for yeari, yearval in enumerate( yearlist):
    print( "Year: " + yearval)
    for filei, fileval in enumerate( filelist[ yeari]):
        print( "File: " + fileval)

        # grab the limits for this case
        # simplified filename
        date = fileval[7:11]
        eyewall_limits = eyewall_finder_helper( yearval, date, metadata)
                    
        # do this for each of the eyewall limit pairs! Can have multiple eyes per crl dataset
        for eyei, eyeval in enumerate( eyewall_limits):
            print("\nEye pass " + str(eyei))
            
            # load crl data
            os.chdir( crl_root_path + yearval)
            crl_data = xr.open_dataset( fileval)

            if len( eyeval) > 0:
                # find the corresponding indices to the time limits
                ind0 = np.argmin( np.abs(crl_data.time.values - eyeval[0] ))
                ind1 = np.argmin( np.abs(crl_data.time.values - eyeval[1] ))

                # clip relevant fields down to the eyewall limits
                H = crl_data.height
                power = crl_data.P_ch1[ ind0 : ind1, :]
                axis = crl_data.time[ ind0 : ind1]
                p3_height = crl_data.p3_height[ ind0 : ind1]
                # find cloud top heights for values within the specified eye distance range
                if yearval == '2021':
                    cutoff = -30
                elif yearval == '2022':
                    cutoff = -40
                heights, time = find_cloud_tops.find_cloud_heights( H, power, axis, p3_height, cutoff_power = cutoff)

                x_mean=np.mean( heights)
                x_std=np.std( heights)
                x_norm = ( heights - x_mean) / x_std
                xN = len( x_norm)
                xsigma = np.std( x_norm)  ## calculate the standard deviation
                xmean = np.mean( x_norm)  ## calculate the mean
                lag=1
                x_norm_pandas = xr.DataArray( x_norm).to_pandas()
                x_t1_m = x_norm_pandas.iloc[0 : -1 * lag] - xmean
                x_t2_m = x_norm_pandas.iloc[ lag : ] - xmean
                xAR1 = method1( x_t1_m, x_t2_m, xN, lag, xsigma)
                Nstar_wilks= np.round(((1-xAR1)/(1+xAR1))*xN) ## Barnes Chapter 2 eq. 88
                Nstar_leith= np.round((-0.5*np.log(xAR1))*xN) ## Barnes Chapter 2 eq. 90
                xNstar = Nstar_wilks
                        
                ## Calculate the power spectrum of red noise with lag1_r to use for significance testing
                # do this for non-smoothed data
                data = heights.copy() # temp, wv
                dists = np.arange(len(data)) * scaling
                
                ### step 1: calculate lag-1 autocorrelation (lag1_r, rho) and the associated p value (lag1_p)
                lag1_r,lag1_p = stats.pearsonr(data[0:len(data)-1],data[1:len(data)])
                ### step 2: Calculate e-folding time for a red-noise process with this lag-1 autocorrelation
                Te = -1./np.log(lag1_r) # After Hartman 6.62 with delta t = 1

                ## Calculate the power spectrum of red noise with lag1_r to use for significance testing
                wtype = 'hamming'
                filttype='butter'
                normalize=True
                window_length = len(heights) / 1.2 # 240 # settings
                T2 = window_length / 2
                freq_w = ( np.arange(0., T2 + 1.) / window_length ) / scaling 

                # filter the data
                # specify the window length for filters
                window = 25 ## (default 25) 
                cutoff = .05 ## (default 1./11.)
                N = 3 ## order
                frequency_cutoff=0.04
                Wn = frequency_cutoff*2 ## scalar given the critical frequency (all higher frequencies are removed)
                # b, a = signal.butter(N, Wn, btype='high')                    
                # y2 = signal.filtfilt(b,a,heights)

                # calculate degrees of freedom n* here!
                dof_welch=len(data) / (window_length / 2)  ### Barnes Eq. 59, factor fw=1 here (based on HW 4 instructions)

                
                # note: I think that method 1 is the correct way to calculate n* here! The window shouldn't impact the 
                # degrees of freedom
                print("Number of Heights = " + str(len(heights)))
                print('Method 1: normalized heights')
                print(f'np.correlate AR1: {round(xAR1[0], 5)}')
                print(f'Effective Sample Size Wilks N* (#independent samples): {Nstar_wilks[0]}')
                nstar_total += Nstar_wilks[0]
                passcount += 1
                autocorrelation_sum += xAR1[0]
            
                print("\nMethod 2: using windows")
                print('lag-1 autocorrelation =',round(lag1_r,2),'and Te =',round(Te,0))
                print("Window = " + str(window_length))
                print('DOF =', dof_welch)

print("\nTotal passes = " + str(passcount))
print("Average autocorrelation = " + str(autocorrelation_sum / passcount))
print("Total n* from individual eye passes = " + str(nstar_total))

Number of data files to be used in analysis: 35
Year: 2021
File: P3_20210811H1_processed.nc

Eye pass 0
File: P3_20210812H1_processed.nc

Eye pass 0
Number of Heights = 343
Method 1: normalized heights
np.correlate AR1: 0.77271
Effective Sample Size Wilks N* (#independent samples): 44.0

Method 2: using windows
lag-1 autocorrelation = 0.77 and Te = 4.0
Window = 285.83333333333337
DOF = 2.3999999999999995

Eye pass 1
Number of Heights = 218
Method 1: normalized heights
np.correlate AR1: 0.93583
Effective Sample Size Wilks N* (#independent samples): 7.0

Method 2: using windows
lag-1 autocorrelation = 0.94 and Te = 16.0
Window = 181.66666666666669
DOF = 2.4

Eye pass 2
Number of Heights = 378
Method 1: normalized heights
np.correlate AR1: 0.98347
Effective Sample Size Wilks N* (#independent samples): 3.0

Method 2: using windows
lag-1 autocorrelation = 0.99 and Te = 110.0
Window = 315.0
DOF = 2.4
File: P3_20210812H2_processed.nc

Eye pass 0
Number of Heights = 374
Method 1: normalized he

Number of Heights = 129
Method 1: normalized heights
np.correlate AR1: 0.60959
Effective Sample Size Wilks N* (#independent samples): 31.0

Method 2: using windows
lag-1 autocorrelation = 0.61 and Te = 2.0
Window = 107.5
DOF = 2.4
File: P3_20210927H1_processed.nc

Eye pass 0
Number of Heights = 118
Method 1: normalized heights
np.correlate AR1: 0.94182
Effective Sample Size Wilks N* (#independent samples): 4.0

Method 2: using windows
lag-1 autocorrelation = 0.94 and Te = 16.0
Window = 98.33333333333334
DOF = 2.4

Eye pass 1
Number of Heights = 77
Method 1: normalized heights
np.correlate AR1: 0.94827
Effective Sample Size Wilks N* (#independent samples): 2.0

Method 2: using windows
lag-1 autocorrelation = 0.94 and Te = 15.0
Window = 64.16666666666667
DOF = 2.4
File: P3_20210929H2_processed.nc

Eye pass 0
Number of Heights = 181
Method 1: normalized heights
np.correlate AR1: 0.88949
Effective Sample Size Wilks N* (#independent samples): 11.0

Method 2: using windows
lag-1 autocorrelat

Number of Heights = 102
Method 1: normalized heights
np.correlate AR1: 0.60696
Effective Sample Size Wilks N* (#independent samples): 25.0

Method 2: using windows
lag-1 autocorrelation = 0.61 and Te = 2.0
Window = 85.0
DOF = 2.4

Eye pass 4
Number of Heights = 58
Method 1: normalized heights
np.correlate AR1: 0.69247
Effective Sample Size Wilks N* (#independent samples): 11.0

Method 2: using windows
lag-1 autocorrelation = 0.68 and Te = 3.0
Window = 48.333333333333336
DOF = 2.4
File: P3_20220920H1_processed.nc

Eye pass 0
Number of Heights = 36
Method 1: normalized heights
np.correlate AR1: 0.86644
Effective Sample Size Wilks N* (#independent samples): 3.0

Method 2: using windows
lag-1 autocorrelation = 0.84 and Te = 6.0
Window = 30.0
DOF = 2.4

Eye pass 1
Number of Heights = 14
Method 1: normalized heights
np.correlate AR1: 0.75988
Effective Sample Size Wilks N* (#independent samples): 2.0

Method 2: using windows
lag-1 autocorrelation = 0.78 and Te = 4.0
Window = 11.66666666666666

C:\Users\etmu9498\AppData\Local\Temp\ipykernel_24428\1532473198.py:90: RuntimeWarning: invalid value encountered in log
  Nstar_leith= np.round((-0.5*np.log(xAR1))*xN) ## Barnes Chapter 2 eq. 90
C:\Users\etmu9498\AppData\Local\Temp\ipykernel_24428\1532473198.py:101: RuntimeWarning: invalid value encountered in log
  Te = -1./np.log(lag1_r) # After Hartman 6.62 with delta t = 1


Number of Heights = 72
Method 1: normalized heights
np.correlate AR1: 0.20156
Effective Sample Size Wilks N* (#independent samples): 48.0

Method 2: using windows
lag-1 autocorrelation = 0.2 and Te = 1.0
Window = 60.0
DOF = 2.4
File: P3_20220925H1_processed.nc

Eye pass 0
Number of Heights = 89
Method 1: normalized heights
np.correlate AR1: 0.72506
Effective Sample Size Wilks N* (#independent samples): 14.0

Method 2: using windows
lag-1 autocorrelation = 0.72 and Te = 3.0
Window = 74.16666666666667
DOF = 2.4

Eye pass 1
Number of Heights = 76
Method 1: normalized heights
np.correlate AR1: 0.74745
Effective Sample Size Wilks N* (#independent samples): 11.0

Method 2: using windows
lag-1 autocorrelation = 0.75 and Te = 4.0
Window = 63.333333333333336
DOF = 2.4
File: P3_20220926H1_processed.nc

Eye pass 0
Number of Heights = 33
Method 1: normalized heights
np.correlate AR1: 0.44573
Effective Sample Size Wilks N* (#independent samples): 13.0

Method 2: using windows
lag-1 autocorrelation 

In [33]:
# test #2- find autocorrelation for all eye clouds combined

# choose which data subset to look at (all, 2021, 2022, or a dictionary)
tc = 'all'

indict = {}
indict['2021'] = ['P3_20210929H2_processed.nc'] # ['P3_20210929H2_processed.nc']
# tc = indict

eye_limits='default'
crl_root_path = "/Users/etmu9498/research/data/crl-all-data-processed/"

# recursively save dates, cloud heights, and power outputs to lists here!
dates, passes, cloudheights, heightaxes, powers = [], [], [], [], []

# use a helper fn to get the relevant years and files
yearlist, filelist = helper_fns_winter2023.get_crl_datasets( tc=tc)

# print out the number of files to be used
filecount = 0
for yeari in range( len( filelist)):
    # count all the names in this year, and add to the count
    filecount += len( filelist[ yeari])
print("Number of data files to be used in analysis: " + str( filecount))

# load eyewall limits from helper function
metadata = eyewall_metadata.all_metadata( eye_limits=eye_limits)

# do this for all the datasets! years and filenames
for yeari, yearval in enumerate( yearlist):
    print( "Year: " + yearval)
    for filei, fileval in enumerate( filelist[ yeari]):
        print( "File: " + fileval)

        # grab the limits for this case
        # simplified filename
        date = fileval[7:11]
        # check if this date exists... if not, give it some empty eyewall limits!
        # also account for fred am and pm cases!!
        if date == '0812':
            if fileval[11:13] == "H1":
                eyewall_limits = metadata[ yearval]['eyewall_limits'][ '0812am']
            elif fileval[11:13] == "H2":
                eyewall_limits = metadata[ yearval]['eyewall_limits'][ '0812pm']
        elif date in metadata[ yearval]['eyewall_limits'].keys():
            eyewall_limits = metadata[ yearval]['eyewall_limits'][ date]
        else:
            eyewall_limits = [ ()]
            
            
        # do this for each of the eyewall limit pairs! Can have multiple eyes per crl dataset
        for eyei, eyeval in enumerate( eyewall_limits):
            # load crl data
            os.chdir( crl_root_path + yearval)
            crl_data = xr.open_dataset( fileval)

            if len( eyeval) > 0:

                # find the corresponding indices to the time limits
                ind0 = np.argmin( np.abs(crl_data.time.values - eyeval[0] ))
                ind1 = np.argmin( np.abs(crl_data.time.values - eyeval[1] ))

                # clip relevant fields down to the eyewall limits
                H = crl_data.height
                power = crl_data.P_ch1[ ind0 : ind1, :]
                axis = crl_data.time[ ind0 : ind1]
                p3_height = crl_data.p3_height[ ind0 : ind1]

                #print( "h: " + str( len( H)))
                #print( "power: " + str( np.shape( power)))
                #print( 't: ' + str( len( axis)))

                # find cloud top heights for values within the specified eye distance range
                if yearval == '2021':
                    cutoff = -30
                elif yearval == '2022':
                    cutoff = -40
                heights, time = find_cloud_tops.find_cloud_heights( H, power, axis, p3_height, cutoff_power = cutoff)
                
                cloudheights.append(heights)
                dates.append(date)
                passes.append(eyei)
                powers.append(power)
                heightaxes.append(H.values)
                
                print("heights found")

# concatenate and plot all cloud heights together!
allheights = np.array([])
for chi, chval in enumerate(cloudheights):
    allheights = np.concatenate( (allheights, chval), axis=0)
print('number of cloud height data points: ' + str(np.shape(allheights)))

# standardize the data and find the autocorrelation!
x_mean=np.mean( allheights)
x_std=np.std( allheights)
x_norm = ( allheights - x_mean) / x_std
xN = len( x_norm)
xsigma = np.std( x_norm)  ## calculate the standard deviation
xmean = np.mean( x_norm)  ## calculate the mean
lag=1
x_norm_pandas = xr.DataArray( x_norm).to_pandas()
x_t1_m = x_norm_pandas.iloc[0 : -1 * lag] - xmean
x_t2_m = x_norm_pandas.iloc[ lag : ] - xmean
xAR1 = method1( x_t1_m, x_t2_m, xN, lag, xsigma)
Nstar_wilks= np.round(((1-xAR1)/(1+xAR1))*xN) ## Barnes Chapter 2 eq. 88
Nstar_leith= np.round((-0.5*np.log(xAR1))*xN) ## Barnes Chapter 2 eq. 90
xNstar = Nstar_wilks

print('Method 1: normalized heights')
print(f'np.correlate AR1: {round(xAR1[0], 5)}')
print(f'Effective Sample Size Wilks N* (#independent samples): {Nstar_wilks[0]}')
nstar_total += Nstar_wilks[0]


Number of data files to be used in analysis: 35
Year: 2021
File: P3_20210811H1_processed.nc
File: P3_20210812H1_processed.nc
heights found
heights found
heights found
File: P3_20210812H2_processed.nc
heights found
heights found
File: P3_20210813H1_processed.nc
heights found
heights found
File: P3_20210816H1_processed.nc
heights found
heights found
File: P3_20210817H1_processed.nc
heights found
heights found
heights found
File: P3_20210818H1_processed.nc
heights found
heights found
heights found
File: P3_20210819H1_processed.nc
heights found
heights found
heights found
File: P3_20210820H1_processed.nc
heights found
heights found
File: P3_20210821H2_processed.nc
heights found
heights found
heights found
File: P3_20210827H1_processed.nc
heights found
heights found
heights found
heights found
File: P3_20210828H1_processed.nc
heights found
File: P3_20210829H1_processed.nc
heights found
File: P3_20210925H1_processed.nc
File: P3_20210926H1_processed.nc
heights found
heights found
heights foun